In [6]:
import pandas as pd
import re
import matplotlib.pyplot as plt

In [8]:
%matplotlib inline

In [9]:
summary = pd.read_csv("atusdata/atussum_2013/atussum_2013.dat")
summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11385 entries, 0 to 11384
Columns: 413 entries, tucaseid to t500107
dtypes: float64(1), int64(412)
memory usage: 36.0 MB


In [10]:
summary.columns

Index(['tucaseid', 'TUFINLWGT', 'TRYHHCHILD', 'TEAGE', 'TESEX', 'PEEDUCA', 'PTDTRACE', 'PEHSPNON', 'GTMETSTA', 'TELFS', 'TEMJOT', 'TRDPFTPT', 'TESCHENR', 'TESCHLVL', 'TRSPPRES', 'TESPEMPNOT', 'TRERNWA', 'TRCHILDNUM', 'TRSPFTPT', 'TEHRUSLT', 'TUDIARYDAY', 'TRHOLIDAY', 'TRTEC', 'TRTHH', 't010101', 't010102', 't010201', 't010299', 't010301', 't010399', 't010401', 't020101', 't020102', 't020103', 't020104', 't020199', 't020201', 't020202', 't020203', 't020299', 't020301', 't020302', 't020303', 't020399', 't020401', 't020402', 't020499', 't020501', 't020502', 't020599', 't020601', 't020602', 't020699', 't020701', 't020799', 't020801', 't020899', 't020901', 't020902', 't020903', 't020904', 't020905', 't020999', 't029999', 't030101', 't030102', 't030103', 't030104', 't030105', 't030106', 't030108', 't030109', 't030110', 't030111', 't030112', 't030199', 't030201', 't030202', 't030203', 't030299', 't030301', 't030302', 't030303', 't030399', 't030401', 't030402', 't030403', 't030404', 't030405',

Pertinent columns:

* TUFINLWGT - statistical weight of respondent
* TRYHHCHILD - age of youngest child in household
* TEAGE - age of respondent
* TESEX - sex of respondent
* TELFS - working status of respondent
* TRCHILDNUM - number of children in household

In [11]:
summary.head()

,tucaseid,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,...,t181501,t181599,t181601,t181801,t189999,t500101,t500103,t500105,t500106,t500107
0,20130101130004,11899905.662034,12,22,2,40,8,2,1,5,...,0,0,0,0,0,0,0,0,0,0
1,20130101130112,4447638.009513,1,39,1,43,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0
2,20130101130123,10377056.507734,-1,47,2,40,1,2,1,4,...,25,0,0,0,0,0,0,0,0,0
3,20130101130611,7731257.992805,-1,50,2,40,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,20130101130616,4725269.227067,-1,45,2,40,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0


In [12]:
adults_crit = (summary.TEAGE >= 18)
no_children_crit = (summary.TRCHILDNUM == 0)

In [13]:
adults_with_no_children = summary[adults_crit & no_children_crit]
adults_with_no_children.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6481 entries, 2 to 11381
Columns: 413 entries, tucaseid to t500107
dtypes: float64(1), int64(412)
memory usage: 20.5 MB


In [14]:
data = adults_with_no_children[['TUFINLWGT', 't120303']]
data = data.rename(columns={"TUFINLWGT": "weight", "t120303": "minutes"})
data.head()

,weight,minutes
2,10377056.507734,60
3,7731257.992805,65
4,4725269.227067,90
5,2372791.046351,270
6,5671341.270490,244


In [15]:
data['weighted_minutes'] = data.weight * data.minutes

In [16]:
data.head()

,weight,minutes,weighted_minutes
2,10377056.507734,60,6.226234e+08
3,7731257.992805,65,5.025318e+08
4,4725269.227067,90,4.252742e+08
5,2372791.046351,270,6.406536e+08
6,5671341.270490,244,1.383807e+09


In [17]:
data.weighted_minutes.sum() / data.weight.sum() / 60

3.1709004734759403

In [18]:
def average_minutes(data, activity_code):
    activity_col = "t{}".format(activity_code)
    data = data[['TUFINLWGT', activity_col]]
    data = data.rename(columns={"TUFINLWGT": "weight", activity_col: "minutes"})
    data['weighted_minutes'] = data.weight * data.minutes
    return data.weighted_minutes.sum() / data.weight.sum()

In [19]:
sleeping = average_minutes(adults_with_no_children, "010101")
sleepless = average_minutes(adults_with_no_children, "010102")
(sleeping + sleepless) / 60

8.7508537061809992

In [20]:
def activity_columns(data, activity_code):
    """For the activity code given, return all columns that fall under that activity."""
    col_prefix = "t{}".format(activity_code)
    return [column for column in data.columns if re.match(col_prefix, column)]

In [21]:
activity_columns(summary, "0101")

['t010101', 't010102']

In [22]:
def average_minutes2(data, activity_code):
    cols = activity_columns(data, activity_code)
    activity_data = data[cols]
    activity_sums = activity_data.sum(axis=1)
    data = data[['TUFINLWGT']]
    data['minutes'] = activity_sums
    data = data.rename(columns={"TUFINLWGT": "weight"})
    data['weighted_minutes'] = data.weight * data.minutes
    return data.weighted_minutes.sum() / data.weight.sum()

In [23]:
average_minutes2(adults_with_no_children, "0101") / 60

-c:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


8.750853706181001

In [24]:
average_minutes2(adults_with_no_children, "01") / 60

9.5541911543273592

In [26]:
group = summary.groupby(["TEAGE", "TESEX", "TRCHILDNUM"])

In [27]:
group.mean().xs(15).xs(1)

,tucaseid,TUFINLWGT,TRYHHCHILD,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,TEMJOT,TRDPFTPT,...,t181501,t181599,t181601,t181801,t189999,t500101,t500103,t500105,t500106,t500107
TRCHILDNUM,,,,,,,,,,,,,,,,,,,,,
1,2.013063e+13,10279004.678881,15.000000,35.157895,1.842105,1.842105,1.157895,4.789474,-0.842105,-0.842105,...,0,0,0,0,1.315789,20.157895,11.052632,0,0,0
2,2.013066e+13,8094486.932688,11.733333,35.133333,1.133333,1.800000,1.066667,4.666667,-0.800000,-0.800000,...,3,0,0,0,0.333333,1.466667,0.000000,0,0,0
3,2.013056e+13,13751422.077139,7.000000,34.846154,1.923077,1.692308,1.230769,4.384615,-0.538462,-0.538462,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,0
4,2.013050e+13,11292383.376507,5.333333,33.666667,1.000000,2.000000,1.333333,3.666667,-0.333333,0.000000,...,0,0,0,0,0.000000,0.000000,0.000000,0,10,0
6,2.013081e+13,6890966.927239,6.000000,34.000000,1.000000,2.000000,2.000000,1.000000,2.000000,1.000000,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,0


## Joining files

In [31]:
respondents = pd.read_csv("atusdata/atusresp_2013/atusresp_2013.dat")
activities = pd.read_csv("atusdata/atusact_2013/atusact_2013.dat")

In [32]:
respondents.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11385 entries, 0 to 11384
Columns: 175 entries, TUCASEID to TXTONHH
dtypes: float64(1), int64(172), object(2)
memory usage: 15.3+ MB


In [33]:
activities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215576 entries, 0 to 215575
Data columns (total 31 columns):
TUCASEID        215576 non-null int64
TUACTIVITY_N    215576 non-null int64
TEWHERE         215576 non-null int64
TRTCCTOT_LN     215576 non-null int64
TRTCC_LN        215576 non-null int64
TRTCOC_LN       215576 non-null int64
TRTEC_LN        215576 non-null int64
TRTHH_LN        215576 non-null int64
TRTNOHH_LN      215576 non-null int64
TRTOHH_LN       215576 non-null int64
TRTONHH_LN      215576 non-null int64
TRTO_LN         215576 non-null int64
TRWBELIG        215576 non-null int64
TUACTDUR        215576 non-null int64
TUACTDUR24      215576 non-null int64
TUCC5           215576 non-null int64
TUCC5B          215576 non-null int64
TUCC7           215576 non-null int64
TUCC8           215576 non-null int64
TUCUMDUR        215576 non-null int64
TUCUMDUR24      215576 non-null int64
TUDURSTOP       215576 non-null int64
TUEC24          215576 non-null int64
TUSTARTTIM     

In [34]:
respondents.head()

,TUCASEID,TULINENO,TUYEAR,TUMONTH,TEABSRSN,TEERN,TEERNH1O,TEERNH2,TEERNHRO,TEERNHRY,...,TXSPEMPNOT,TXSPUHRS,TXTCC,TXTCCTOT,TXTCOC,TXTHH,TXTNOHH,TXTO,TXTOHH,TXTONHH
0,20130101130004,1,2013,1,-1,-1,-1,-1,-1,-1,...,-1,-1,0,0,0,0,0,-1,-1,-1
1,20130101130112,1,2013,1,-1,-1,-1,-1,-1,2,...,0,0,0,0,0,0,-1,0,0,-1
2,20130101130123,1,2013,1,-1,-1,-1,-1,-1,-1,...,0,0,-1,0,0,-1,-1,-1,-1,-1
3,20130101130611,1,2013,1,-1,-1,-1,-1,-1,2,...,-1,-1,-1,0,0,-1,-1,-1,-1,-1
4,20130101130616,1,2013,1,-1,-1,-1,-1,-1,2,...,-1,-1,-1,0,0,-1,-1,-1,-1,-1


In [35]:
activities.head()

,TUCASEID,TUACTIVITY_N,TEWHERE,TRTCCTOT_LN,TRTCC_LN,TRTCOC_LN,TRTEC_LN,TRTHH_LN,TRTNOHH_LN,TRTOHH_LN,...,TUDURSTOP,TUEC24,TUSTARTTIM,TUSTOPTIME,TUTIER1CODE,TUTIER2CODE,TUTIER3CODE,TRCODE,TRTIER2,TXWHERE
0,20130101130004,1,-1,0,0,0,-1,0,0,-1,...,2,-1,04:00:00,12:00:00,1,1,1,10101,101,0
1,20130101130004,2,1,5,5,0,-1,5,5,-1,...,1,-1,12:00:00,12:05:00,11,1,1,110101,1101,0
2,20130101130004,3,1,120,120,0,-1,120,120,-1,...,1,-1,12:05:00,14:05:00,12,3,3,120303,1203,0
3,20130101130004,4,1,0,0,0,-1,0,0,-1,...,2,-1,14:05:00,19:00:00,6,3,1,60301,603,0
4,20130101130004,5,1,30,30,0,-1,30,30,-1,...,1,-1,19:00:00,19:30:00,11,1,1,110101,1101,0


In [41]:
merged = pd.merge(respondents, activities, left_on="TUCASEID", right_on="TUCASEID")
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215576 entries, 0 to 215575
Columns: 205 entries, TUCASEID to TXWHERE
dtypes: float64(1), int64(200), object(4)
memory usage: 338.8+ MB


In [42]:
merged.head()

,TUCASEID,TULINENO,TUYEAR,TUMONTH,TEABSRSN,TEERN,TEERNH1O,TEERNH2,TEERNHRO,TEERNHRY,...,TUDURSTOP,TUEC24,TUSTARTTIM,TUSTOPTIME,TUTIER1CODE,TUTIER2CODE,TUTIER3CODE,TRCODE,TRTIER2,TXWHERE
0,20130101130004,1,2013,1,-1,-1,-1,-1,-1,-1,...,2,-1,04:00:00,12:00:00,1,1,1,10101,101,0
1,20130101130004,1,2013,1,-1,-1,-1,-1,-1,-1,...,1,-1,12:00:00,12:05:00,11,1,1,110101,1101,0
2,20130101130004,1,2013,1,-1,-1,-1,-1,-1,-1,...,1,-1,12:05:00,14:05:00,12,3,3,120303,1203,0
3,20130101130004,1,2013,1,-1,-1,-1,-1,-1,-1,...,2,-1,14:05:00,19:00:00,6,3,1,60301,603,0
4,20130101130004,1,2013,1,-1,-1,-1,-1,-1,-1,...,1,-1,19:00:00,19:30:00,11,1,1,110101,1101,0


In [43]:
month_group = respondents.groupby(["TUMONTH"])
monthly_averages = month_group.mean()
average_minutes2(monthly_averages, "01")